# Prospect + specutils + SPARCL

Project: Obtain DESI EDR spectra using the [NOIRLab SPARCL spectrum service](https://astrosparcl.datalab.noirlab.edu), convert data to [specutils objects](https://specutils.readthedocs.io/en/stable/) as needed, and use [prospect](https://desi-prospect.readthedocs.io/en/latest/) to display the data.

## Imports

Note that we are not relying heavily on the DESI software stack.

In [ ]:
import os
import sys
# sys.path.insert(0, os.path.join(os.environ['HOME'], 'Documents', 'Code', 'git', 'desihub', 'prospect', 'py'))
import numpy as np
import astropy.units as u
from astropy.table import QTable
from specutils import __version__ as specutils_version
from prospect import __version__ as prospect_version
from prospect.viewer import plotspectra
from prospect.specutils import Spectra
from sparcl import __version__ as sparcl_version
import sparcl.client
from dl import version as dl_version, queryClient as qc, storeClient as sc, specClient as spec
print(f"astro-datalab=={dl_version}")
print(f"specutils=={specutils_version}")
print(f"prospect=={prospect_version}")
print(f"sparcl=={sparcl_version}")

## Start SPARCL Client

In [ ]:
client = sparcl.client.SparclClient(url='https://sparclstage.datalab.noirlab.edu') # using the STAGE machine
client

## Set up Data Lab database interface

In [ ]:
qc.set_profile('db01')

## Find DESI spectra

SPARCL provides access to DESI spectra that have been coadded by HEALPixel.  This corresponds to entries in the `desi_edr.zpix` table.

In [ ]:
q = """SELECT z.targetid, z.chi2, z.z, z.zerr, z.zwarn, z.spectype, z.subtype, z.coadd_numexp, z.coadd_exptime, z.healpix, z.deltachi2
FROM desi_edr.zpix AS z
WHERE z.zcat_primary AND z.survey = 'sv3' AND z.program = 'dark' AND z.spectype = 'GALAXY' AND z.z BETWEEN 0.5 AND 0.9 AND z.zwarn = 0 ORDER BY z.targetid LIMIT 50;
"""
result = qc.query(sql=q, fmt='table')
result

**QA**: Do we really have 50 unique spectra?

In [ ]:
assert (np.unique(result['targetid']) == result['targetid']).all()

## Retrieve DESI spectra

With the set of `targetid` obtained above, we can directly retrieve DESI spectra.

In [ ]:
include = client.get_all_fields(dataset_list=['DESI-EDR'])
spectra = client.retrieve_by_specid(result['targetid'].value.tolist(), include=include, dataset_list=['DESI-EDR'], verbose=True)
spectra.info

**QA**: Did we really find all of the expected spectra?

In [ ]:
assert (np.array([r.targetid for r in sorted(spectra.records, key=lambda x: x.targetid)]) == result['targetid']).all()
assert all([r.survey == 'sv3' for r in spectra.records])
assert all([r.program == 'dark' for r in spectra.records])

In [ ]:
spectra.records[0]

## Organize metadata

Prospect needs several inputs:

1. An object containing spectra.  In this case we'll use [`prospect.specutils.Spectra`](https://desi-prospect.readthedocs.io/en/latest/api.html#prospect.specutils.Spectra), which inherits from [`SpectrumList`](https://specutils.readthedocs.io/en/stable/api/specutils.SpectrumList.html#specutils.SpectrumList), and is really just a [`Spectrum1D`](https://specutils.readthedocs.io/en/stable/api/specutils.Spectrum1D.html#specutils.Spectrum1D) object underneath.
   * The object contains the usual flux, wavelength, uncertainty.
   * In addtion a "fibermap" table is needed. This should be an Astropy `Table` with the expected columns.
2. A redshift catalog. This should be an Astropy `Table` with the expected columns.
3. A model spectrum.  The model is actually provided by SPARCL, but we need to input it separately.

### Spectrum object

First we assemble the components of the spectrum object

In [ ]:
flux = np.zeros((len(spectra.records), spectra.records[0].flux.shape[0]), dtype=spectra.records[0].flux.dtype)
uncertainty = np.zeros((len(spectra.records), spectra.records[0].ivar.shape[0]), dtype=spectra.records[0].ivar.dtype)
mask = np.zeros((len(spectra.records), spectra.records[0].mask.shape[0]), dtype=spectra.records[0].mask.dtype)
meta = {'sparcl_id': list(), 'data_release': list()}
sparcl_id = list()
data_release = list()
for k in range(len(spectra.records)):
    flux[k, :] = spectra.records[k].flux
    uncertainty[k, :] = spectra.records[k].ivar
    mask[k, :] = spectra.records[k].mask
    meta['sparcl_id'].append(spectra.records[k].sparcl_id)
    meta['data_release'].append(spectra.records[k].data_release)

And the "fibermap" table. We'll start with photometric quantities.

In [ ]:
columns = ('targetid', 'ra', 'dec', 'ref_epoch', 'pmra', 'pmdec', 'ebv', 'flux_g', 'flux_r', 'flux_z', 'flux_w1', 'flux_w2')
q = """SELECT {0}
FROM desi_edr.photometry
WHERE targetid IN ({1}) ORDER BY targetid;
""".format(', '.join(columns), ', '.join([str(t) for t in result['targetid'].value.tolist()]))
fibermap = qc.query(sql=q, fmt='table')
for col in fibermap.colnames:
    if col == 'ra' or col == 'dec':
        fibermap.rename_column(col, 'TARGET_' + col.upper())
    else:
        fibermap.rename_column(col, col.upper())

**QA**: Did we find photometry for every `targetid`?

In [ ]:
assert (fibermap['TARGETID'] == result['targetid']).all()

Next we add targeting bitmasks.

In [ ]:
columns = ('targetid', 'sv3_desi_target', 'sv3_bgs_target', 'sv3_mws_target', 'sv3_scnd_target')
q = """SELECT DISTINCT {0}
FROM desi_edr.target
WHERE targetid IN ({1}) ORDER BY targetid;
""".format(', '.join(columns), ', '.join([str(t) for t in fibermap['TARGETID'].value.tolist()]))
targeting = qc.query(sql=q, fmt='table')
for col in targeting.colnames:
    targeting.rename_column(col, col.upper())

**QA**: Did we find targeting for every `targetid`?

In [ ]:
assert (targeting['TARGETID'] == fibermap['TARGETID']).all()

Add columns into `fibermap`.

In [ ]:
for col in ('SV3_DESI_TARGET', 'SV3_BGS_TARGET', 'SV3_MWS_TARGET', 'SV3_SCND_TARGET'):
    fibermap.add_column(targeting[col])
fibermap

Finally assemble the object.

In [ ]:
prospect_spectra = Spectra(bands=['coadd'],
                           wave={'coadd': spectra.records[0].wavelength},
                           flux={'coadd': flux},
                           ivar={'coadd': uncertainty},
                           mask={'coadd': mask},
                           fibermap=fibermap,
                           meta={'coadd': meta})

### Redshift catalog

We can re-use the initial query above; it was deliberately constructed.

In [ ]:
zcatalog = result.copy()
for col in zcatalog.colnames:
    if col == 'healpix':
        zcatalog.rename_column(col, 'HPXPIXEL')
    else:
        zcatalog.rename_column(col, col.upper())
zcatalog

### Model spectra

Prospect expects a model in the form of a tuple containing wavelength and flux. Since SPARCL provides the model, this is easy.  There are other ways to specify the model, but these require more access to the DESI software stack *and* data *files*.

In [ ]:
model_flux = np.zeros((len(spectra.records), spectra.records[0].model.shape[0]), dtype=spectra.records[0].model.dtype)
for k in range(len(spectra.records)):
        model_flux[k, :] = spectra.records[k].model
model = (spectra.records[0].wavelength, model_flux)

## Start prospect

With everything assembled, the interface to prospect is just a single call.

In [ ]:
plotspectra(prospect_spectra, zcatalog=zcatalog, redrock_cat=None, notebook=True, with_thumb_tab=False, with_vi_widgets=False, with_coaddcam=False, mask_type='SV3_DESI_TARGET',
            model_from_zcat=False, model=model)

## TODO: Also display SDSS/BOSS/eBOSS spectra obtained from SPARCL

Old stuff below this point.

## SDSS/eBOSS spPlate file

SDSS spectra are stored per-plate in spPlate files.  These contain 640 spectra for the original SDSS spectrograph or 1000 spectra for the BOSS/eBOSS spectrograph.  All spPlate files have a common wavelength solution, so a spPlate file can be represented by a Spectrum1D object.

In [ ]:
# os.environ['SPECTRO_REDUX'] = os.path.join(os.environ['HOME'], 'Documents', 'Data', 'sdss', 'dr16', 'sdss', 'spectro', 'redux')
os.environ['SPECTRO_REDUX'] = 'sdss_dr16://' + os.path.join('sdss', 'spectro', 'redux')
run2d = '26'
plate = '2955'
mjd = '54562'
sdss_spectra = os.path.join(os.environ['SPECTRO_REDUX'], run2d, plate, f'spPlate-{plate}-{mjd}.fits')
sdss_redshifts = os.path.join(os.environ['SPECTRO_REDUX'], run2d, plate, f'spZbest-{plate}-{mjd}.fits')
print(sdss_spectra)
print(sdss_redshifts)

In [ ]:
run2d = 'v5_13_0'
plate = '9599'
mjd = '58131'
eboss_spectra = os.path.join(os.environ['SPECTRO_REDUX'], run2d, plate, f'spPlate-{plate}-{mjd}.fits')
eboss_redshifts = os.path.join(os.environ['SPECTRO_REDUX'], run2d, plate, run2d, f'spZbest-{plate}-{mjd}.fits')
print(eboss_spectra)
print(eboss_redshifts)

## SDSS Spectra

In [ ]:
%%time
sdss = read_spPlate(sc.get(sdss_spectra, mode='fileobj'), limit=50)
sdss

In [ ]:
%%time
# sdss_spectra_ids = spec.query(plate=plate, ...)
# sdss = spec.getSpec(sdss_spectra_ids, fmt='spectrum1d')
# sdss

In [ ]:
sdss_z, sdss_model = read_spZbest(sc.get(sdss_redshifts, mode='fileobj'), limit=50)

In [ ]:
plotspectra(sdss, zcatalog=sdss_z, model=(sdss_model.spectral_axis.value, sdss_model.flux.value),
            notebook=True, title=os.path.basename(sdss_spectra),
            model_from_zcat=False, with_coaddcam=False, mask_type='PRIMTARGET', with_thumb_tab=False, with_vi_widgets=False)

## eBOSS Spectra

In [ ]:
%%time
eboss = read_spPlate(sc.get(eboss_spectra, mode='fileobj'), limit=50)
eboss_z, eboss_model = read_spZbest(sc.get(eboss_redshifts, mode='fileobj'), limit=50)
eboss

In [ ]:
plotspectra(eboss, zcatalog=eboss_z, model=(eboss_model.spectral_axis.value, eboss_model.flux.value),
            notebook=True, title=os.path.basename(eboss_spectra),
            model_from_zcat=False, with_coaddcam=False, mask_type='EBOSS_TARGET1', with_thumb_tab=False, with_vi_widgets=False)